In [3]:
!pip install chiecthuyenngoaixa

In [5]:
import ctnx
ctnx.remove_diacritics("Bốn mùa")

'Bon mua'

In [21]:
def uri_from_label(label, namespace):
    return  URIRef(namespace + label.replace(" ","_"))

In [24]:
from rdflib.namespace import RDF, SKOS
from rdflib import Graph, Literal, URIRef, Namespace

# Định nghĩa đối tượng Graph của rdflib
g = Graph()
namespace = "https://github.com/viet-ha-pham/data#"

vhd = Namespace(namespace)

# Gán namespace vhd cho đối tượng Graph
g.bind('vhd', vhd)


# Mở tệp input với định dạng outline
with open('E:/tab-indent.txt', 'r', encoding="utf-8") as f:
    # Đọc tệp input và tạo các đối tượng SKOS
    last_indent = 0
    last_concept = None
    for line in f:
        indent = len(line) - len(line.lstrip())
        label = line.strip()
        
        if indent == 0:
            last_concept = uri_from_label(label, namespace)
            g.add((last_concept, RDF.type, SKOS.Concept))
            g.add((last_concept, SKOS.prefLabel, Literal(label)))
        elif indent > last_indent:
            narrower_concept =  uri_from_label(label, namespace)
            g.add((narrower_concept, RDF.type, SKOS.Concept))
            g.add((narrower_concept, SKOS.prefLabel, Literal(label)))
            g.add((last_concept, SKOS.narrower, narrower_concept))
            last_concept = narrower_concept
        elif indent == last_indent:
            narrower_concept =  uri_from_label(label, namespace)
            g.add((narrower_concept, RDF.type, SKOS.Concept))
            g.add((narrower_concept, SKOS.prefLabel, Literal(label)))
            g.add((last_concept, SKOS.narrower, narrower_concept))
            last_concept = narrower_concept
        else:
            for i in range(last_indent - indent):
                last_concept = g.value(last_concept, SKOS.broader)
            narrower_concept = uri_from_label(label, namespace)
            g.add((narrower_concept, RDF.type, SKOS.Concept))
            g.add((narrower_concept, SKOS.prefLabel, Literal(label)))
            if last_concept:
                g.add((last_concept, SKOS.narrower, narrower_concept))
            last_concept = narrower_concept
        last_indent = indent

# Lưu tệp output dưới dạng RDF/XML
g.serialize(destination='E:/result.ttl', format='ttl')

<Graph identifier=N2cb7c551f316410b94c684ac1c31b3ad (<class 'rdflib.graph.Graph'>)>